In [ ]:
import pandas as pd
import numpy as np

%run Util.ipynb
%run Section.ipynb
%run Constant.ipynb
%run DataSource.ipynb
%run DataSourceManager.ipynb

class SectionManager:
    section_df = pd.DataFrame(columns=section_columns)
    day_k = pd.DataFrame()
    week_k = pd.DataFrame()

    # symbol=SZ#002335
    def __init__(self, symbol, level, short, mid):
        self.symbol = symbol
        self.level = level
        self.short = short
        self.mid = mid
        
        self.metric = Metric(self.symbol, self.level, self.short, self.mid)
            
        self.start_date = self.metric.get_stock_start_date()
        self.end_date = self.metric.get_stock_end_date()
        
        self.section_df = pd.DataFrame(columns=section_columns)
        self.section = Section(self.symbol, self.level, self.short, self.mid)
        
    
    def build_file_path(self):
        original_name = self.symbol
        return  original_section_dir + self.level + '/' + self.level+'_'+str(self.short)+'_'+str(self.mid)+'_' 'sections_'+section_version+'_'+ original_name +'.txt'
    
            
    def get_all_cross_sections(self):
        file_path = self.build_file_path()
        
        if(os.path.exists(file_path)):
            self.section_df = pd.read_csv(file_path)
            
            self.section_df['s_datetime'] = pd.to_datetime(self.section_df['s_datetime'])
            self.section_df['e_datetime'] = pd.to_datetime(self.section_df['e_datetime'])
        else:
            print('build '+self.symbol+' sections')
            self.analyse_all_cross_sections()
                    
            file_path = self.build_file_path()
            self.section_df.to_csv(file_path, index=0)

        return self.section_df
        

    def find_sections_between(self, start_date, end_date):
        if(self.section_df.shape[0]==0):
            self.section_df = self.get_all_cross_sections()
            
        start_datetime = to_datetime(start_date)
        end_datetime = to_datetime(end_date)

        part = self.section_df[(self.section_df['s_datetime']>=start_datetime) & (self.section_df['s_datetime']<=end_datetime)]

        return part

    
#     -------------------------- Private Functions -------------------------------------
    def get_highest_price_percent(self,start_date,end_date):
        max_price_k = self.metric.get_max_price_date(start_date,end_date)

        highest_price = max_price_k['high']
        enter_price = self.metric.get_cur_price(start_date)
        
        entry_to_highest_increase_percent = round(100*(highest_price-enter_price)/enter_price,2)

        return entry_to_highest_increase_percent
        
    
    def analyse_cross_for_one_specific_day(self,cur_date):
        if(self.metric.is_up_cross(cur_date)):
            section_start = cur_date
            next_sections = self.section.get_section(section_start, self.end_date)
            previous_sections = self.section.get_section(self.start_date, section_start)

            section_end = self.metric.next_down_cross(next_sections)
            if(section_end == 'None'):
                return 
            
            start_price = self.metric.get_cur_price(section_start)
            end_price = self.metric.get_latest_price(section_end)
            
            next_k_down_mid_ma_date = self.metric.next_down_ma(self.mid, next_sections)
            
            amplitude = self.section.get_amplitude_percent(section_start,section_end)
            ma_percent = self.section.get_ma_increase_per(section_start,section_end)

            price_percent = self.section.get_price_increase_per(section_start,section_end)
            break_mid_ma_price_incre_percent = self.section.get_price_increase_per(section_start,next_k_down_mid_ma_date)
            
            periods = self.section.how_many_periods(section_start,section_end)
            boll = self.metric.is_on_boll(cur_date)
            boll99 = self.metric.is_on_boll99(cur_date)
            
            ma_short_direction = self.metric.ma_direction(cur_date, self.short)
            ma_mid_direction = self.metric.ma_direction(cur_date, self.mid)
            
            max_increase_percent = self.get_highest_price_percent(cur_date,section_end)
                        
            return [self.symbol,section_start,section_end,start_price,end_price,amplitude,ma_percent,
                    periods,boll,price_percent,ma_short_direction,ma_mid_direction,
                   break_mid_ma_price_incre_percent,boll99,max_increase_percent]
    
        
#   找到该股所有的金叉，死叉波段
    def analyse_all_cross_sections(self):
        print('start to analyse '+self.level+' level '+ self.symbol+' all cross sections...')
        all_dates = self.metric.list_stock_all_dates()[self.mid:]
       
        start_datetime = to_datetime(self.start_date)
        end_datetime = to_datetime(self.end_date)

        for date in all_dates:
            if(not(out_of_range(date,start_datetime,end_datetime))):
                one_section = self.analyse_cross_for_one_specific_day(date)
                
                if(not(one_section is None) and (len(one_section)>0)):
                    self.section_df.loc[self.section_df.shape[0] +1] = one_section

        self.section_df['s_datetime'] = pd.to_datetime(self.section_df['s_date'])
        self.section_df['e_datetime'] = pd.to_datetime(self.section_df['e_date'])

        return self.section_df
    
    
    